# IMAGINE tutorial 6
## Personalization and extension

Here we review the design of the IMAGINE pipeline and
demonstrate how to personalise or extend it, quickly designing
each of its components

<img src="https://bitbucket.org/hammurabicode/imagine/wiki/imagine_design.png" alt="IMAGINE" style="width: 800px;"/>

#### Outline (aka advice for quick reading)

* **Theorists/modellers** interested in including/constraining a new model of a quantity previously known to IMAGINE find all they need reading about [Fields and Factories](#Fields-and-Factories).  They may also want to think about their [Fields and Factories](#Priors) while exploring the parameter space.
* **Observers** interested in replacing an existing dataset, may want to jump to [Measurements and covariances](#Measurements-and-covariances)
* If one wants to include an new **observable quantity** or **dependence on a field** they should go throug the previous sections and look at [Simulators](#Observables-and-Observable-dictionaries).
* For any other case, just read through.


## Fields and Factories

Within the IMAGINE pipeline, spatially varying physical quantities are represented by
[Field objects](https://imagine-code.readthedocs.io/en/latest/imagine.fields.html#module-imagine.fields).
This can be a *scalar*, as the number density of thermal electrons, or a *vector*, as the Galactic magnetic field. 



In order to extend or personalise adding in one's own model for an specific field, 
one needs to follow a small number of simple steps:  
1. choose a **coordinate grid** where your model will be evaluated,
2. write a **field class**, and
3. write a **field factory** class.

The **field objects** will do actually computation of the physical field, given a set of physical parameters and a coordinate grid. 
The **field factory objects** take care of book-keeping tasks: e.g. they hold the parameter ranges and default values, and translate the dimensionless parameters used by the sampler (always in the interval $[0,1]$) to physical parameters.

### Coordinate grid

You can create your own coordinate grid by subclassing [`imagine.fields.grid.BaseGrid`](https://imagine-code.readthedocs.io/en/latest/imagine.fields.html#imagine.fields.grid.BaseGrid).  The only thing which has to actually be programmed in the new sub-class is a
method overriding [`generate_coordinates()`](https://imagine-code.readthedocs.io/en/latest/imagine.fields.html#imagine.fields.grid.BaseGrid.generate_coordinates), which produces a dictionary of numpy arrays containing coordinates in *either* cartesian,
cylindrical or spherical coordinates (generally assumed, in Galactic contexts, to be centred in the centre of the Milky Way).

Typically, however, it is sufficient to use a simple grid with coordinates uniformly-spaced in cartesian, spherical or cylindrical 
coordinates. This can be done using the [`UniformGrid`](https://imagine-code.readthedocs.io/en/latest/imagine.fields.html#imagine.fields.grid.UniformGrid) class. `UniformGrid` objects are initialized with the arguments: `box`, which contains the ranges of each coordinate in $\rm kpc$ or $\rm rad$; `resolution`, a list of integers containing the number of grid points on each dimension; and `grid_type`, which can be either 'cartesian' (default), 'cylindrical' or 'spherical'.

In [ ]:
import imagine as img
import numpy as np

# A cartesian grid can be constructed as follows
cartesian_grid = img.UniformGrid(box=[[-15, 15],[-15, 15],[-15, 15]],
                          resolution = [12,12,12])

# For cylindrical grid, the limits are specified assuming 
# the order: r (cylindrical), phi, z
cylindrical_grid = img.UniformGrid(box=[[0.25, 15],[-np.pi,np.pi],[-15,15]],
                      resolution = [9,12,9],
                      grid_type = 'cylindrical')

# For spherical grid, the limits are specified assuming 
# the order: r (spherical), theta, phi (azimuth)
spherical_grid = img.UniformGrid(box=[[0, 15],[0, np.pi], [-np.pi,np.pi],],
                      resolution = [12,10,10],
                      grid_type = 'spherical')

The grid object will produce the grid only when the a coordinate value is first accessed, 
through the properties 'x', 'y','z','r_cylindrical','r_spherical', 'theta' and 'phi'. 

The grid object also takes care of any coordinate conversions that are needed, for example:

In [ ]:
print(spherical_grid.x[5,5,5], cartesian_grid.r_spherical[5,5,5])

In the following figure we illustrate the effects of different choices of 'grid_type' while using UniformGrid. 

(Note that, for plotting purposes, everything is converted to cartesian coordinates)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.scatter(cartesian_grid.x, cartesian_grid.y, color='y', label='Cartesian', alpha=0.5)
plt.scatter(cylindrical_grid.x, cylindrical_grid.y, label='cylindrical', alpha=0.5)
plt.scatter(spherical_grid.x, spherical_grid.y, color='g', label='spherical', alpha=0.5)
plt.xlabel('x'); plt.ylabel('y')
plt.legend()

plt.subplot(1,2,2)
plt.scatter(cartesian_grid.x, cartesian_grid.z, color='y', label='Cartesian', alpha=0.5)
plt.scatter(cylindrical_grid.x, cylindrical_grid.z, label='cylindrical', alpha=0.5)
plt.scatter(spherical_grid.x, spherical_grid.z, label='spherical', color='g', alpha=0.5)
plt.xlabel('x'); plt.ylabel('z')
plt.legend();




### Field objects

As we mentioned before, [`Field`](https://imagine-code.readthedocs.io/en/latest/imagine.fields.html#module-imagine.fields.field) objects handle the calculation of any physical field.

To ensure that your new personalised field is compatible with any simulator,
it needs to be a subclass of one of the [pre-defined field classes](https://imagine-code.readthedocs.io/en/latest/physical_fields.html). Some examples of which are:
 * `MagneticField`
 * `ThermalElectronDensity`
 * `CosmicRayDistribution`

Let us illustrate this by defining a thermal electron number density field 
which decays exponentially with cylindrical radius, $R$, .
$$ n_e(R) = n_{e,0} e^{-R/R_e}$$
This has two parameters: the number density of thermal electrons at the 
centre, $n_{e,0}$, and a scale radius, $R_e$.



In [ ]:
from imagine import ThermalElectronDensityField

class ExponentialThermalElectrons(ThermalElectronDensityField):
    """Example: thermal electron density of an exponential disc"""
    
    field_name = 'exponential_disc_thermal_electrons'
    
    @property
    def field_checklist(self):
        return {'central_density_cm-3' : None, 'scale_radius_kpc' : None }
    
    def get_field(self, parameters):
        R = self.grid.r_cylindrical
        Re = parameters['scale_radius_kpc']
        n0 = parameters['central_density_cm-3']
        
        return n0*np.exp(-np.abs(R/Re))

With these few lines we have created our IMAGINE-compatible™ thermal electron density field class!

The class-attribute `field_name` allows one to keep track of which model we have used to generate our field. 

The `field_checklist` property is a dictionary whose keys are required parameters for this particular kind of field. The values in the dictionary can be used for specialized checking by some simulators (but can be left as `None` in the general case).

The function `get_field` is what actually computes the density field. Note that it can access an associated grid object, which is stored in the `grid` attribute.

Let us now see this at work. First, let us creat an instance of `ExponentialThermalElectrons`. Any Field instance should be initialized 
providing a Grid object and a dictionary of parameters.

In [ ]:
electron_distribution = ExponentialThermalElectrons(
    parameters={'central_density_cm-3': 1.0, 'scale_radius_kpc': 3.3},
    grid=cartesian_grid)

IMAGINE will access the field produced by `cr_distribution` *only using the* `data` attribute. For example:

In [ ]:
electron_distribution.data[3:6,3:6,3:6]

If we now wanted to plot the thermal electron density computed by this as a function of, say, *spherical radius*, $r_{\rm sph}$. This can be done in the following way

In [ ]:
# The spherical radius can be read from the grid object
rspherical = electron_distribution.grid.r_spherical.ravel()
# The electron density is the data array
ne = electron_distribution.data.ravel()

plt.plot(rspherical, ne, '.')
plt.xlabel(r'$r_{\rm sph}\;[\rm kpc]$'); plt.ylabel(r'$n_{\rm cr}\;[\rm cm^{-3}]$');

Let us do another simple field example: a constant magnetic field.

It follows the same basic template.

In [ ]:
from imagine import MagneticField
    
class ConstantMagneticField(MagneticField):
    """Example: constant magnetic field"""
    field_name = 'constantB'
    
    @property
    def field_checklist(self):
        return {'Bx_muG': None, 'By_muG': None, 'Bz_muG': None}
    
    def get_field(self, parameters):
        # Creates an empty array to store the result
        B = np.empty(self.data_shape)        
        # For a magnetic field, the output must be of shape: 
        # (Nx,Ny,Nz,Nc) where Nc is the index of the component. 
        # Computes Bx
        B[:,:,:,0] = parameters['Bx_muG']*np.ones(self.grid.shape)
        # Computes By
        B[:,:,:,1] = parameters['By_muG']*np.ones(self.grid.shape)
        # Computes Bz
        B[:,:,:,2] = parameters['Bz_muG']*np.ones(self.grid.shape)        
        return B

The main difference from the thermal electrons case is that the shape of the final array has to accomodate all the three components of the magnetic field.

As before, we can generate a realisation of this

In [ ]:
p = {'Bx_muG': 0, 'By_muG': 1., 'Bz_muG': 0.5}
B = ConstantMagneticField(parameters=p, grid=cartesian_grid)

And inspect how it went

In [ ]:
r_spherical = B.grid.r_spherical.ravel()

for i, name in enumerate(['x','y','z']):
    plt.plot(r_spherical, B.data[...,i].ravel(), 
             label='$B_{}$'.format(name))
plt.xlabel(r'$r_{\rm sph}\;[\rm kpc]$'); plt.ylabel(r'$B\;[\mu\rm G]$')
plt.legend();



More information about the field can be found inspecting the object

In [ ]:
print('Field type: ', B.field_type)
print('Data shape: ', B.data_shape)
print('What is each axis? Answer:', B.data_description)

### Field factory

Associated with each Field class we need to prepare a FieldFactory class, which will take care (separately) of the scaling of parameter ranges. This is can done through the 
following simple templates

In [ ]:
from imagine import GeneralFieldFactory

class ExponentialThermalElectrons_Factory(GeneralFieldFactory):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.field_class = ExponentialThermalElectrons
        self.default_parameters = {'central_density_cm-3': 1,
                                   'scale_radius_kpc': 3.0}
        self.parameter_ranges = {'central_density_cm-3': [0,10],
                                 'scale_radius_kpc': [0,10]}
        
class ConstantMagneticField_Factory(GeneralFieldFactory):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.field_class = ConstantMagneticField
        self.default_parameters = {'Bx_muG': 0.0,
                                   'By_muG': 0.0,
                                   'Bz_muG': 0.0}
        self.parameter_ranges = {'Bx_muG': [-30, 30],
                                 'By_muG': [-30, 30],
                                 'Bz_muG': [-10, 10]}

We can now create instances of any of these. Which can return Field objects through the 
`generate` method.

In [ ]:
Bfactory = ConstantMagneticField_Factory(grid=cartesian_grid)
newB = Bfactory.generate()
newB.field_name, newB.parameters

The parameters were all set to their default values.
In practice, before the factory object is used, the pipeline 
first sets a list of *active* parameters 
(inactive parameters are keept with their default values)

In [ ]:
Bfactory.active_parameters = ['Bx_muG']

The `generate` method can then be called with a dictionary containing *scaled* *dimensionless* variables as values

In [ ]:
dimensionless_scaled_variables = {'Bx_muG': 0.7}  # This is NOT 0.5 microGauss!
newB = Bfactory.generate(variables=dimensionless_scaled_variables)
newB.parameters

We see that the value corresponds to the 70% of the range between $-30$ and $30\,\mu \rm G$

## Measurements and covariances

'Observable dictionaries' are the way observational and simulated data are organised within IMAGINE. There are three types of Observable dictionaries: `Measurements`, `Simulations` and `Covariances`. 

Data can be generally input data to these using a help `Dataset` class.
Let us illustrate this with the Faraday depth map obtained by Oppermann et al. 2012 ([arXiv:1111.6186](https://arxiv.org/abs/1111.6186)). 

The following snippet will download the data (a ~4MB FITS file) to RAM and opens it.

In [ ]:
import requests, io
import numpy as np
from astropy.io import fits

download = requests.get('https://wwwmpa.mpa-garching.mpg.de/ift/faraday/2012/faraday.fits')
raw_dataset = fits.open(io.BytesIO(download.content))
raw_dataset.info()

Now we will feed this into an IMAGINE `Dataset`. It requires 
converting the data into a proper numpy array of floats. 
To allow this notebook running on a small memory laptop, we
will also reduce the size of the arrays (only taking 1 value every 256).

In [ ]:
from imagine.observables.dataset import FaradayDepthHEALPixDataset

# Adjusts the data to the right format
fd_raw = raw_dataset[3].data.astype(np.float)
sigma_fd_raw = raw_dataset[4].data.astype(np.float)
# Makes it small, to save memory
fd_raw = fd_raw[::256]
sigma_fd_raw = sigma_fd_raw[::256]
# Loads into a Dataset
dset = FaradayDepthHEALPixDataset(data=fd_raw, error=sigma_fd_raw)

To keep things organised and useful, we strongly suggest to create a 
personalised `dataset` class and make it available to the rest of the
consortium in the GitHub repository. An example of such a class is the following

In [ ]:
import requests, io
import numpy as np
from astropy.io import fits
from imagine.observables.dataset import FaradayDepthHEALPixDataset

class FaradayDepthOppermann2012(FaradayDepthHEALPixDataset):
    def __init__(self, skip=None):
        download = requests.get('https://wwwmpa.mpa-garching.mpg.de/ift/faraday/2012/faraday.fits')
        raw_dataset = fits.open(io.BytesIO(download.content))
    
        # Adjusts the data to the right format
        fd_raw = raw_dataset[3].data.astype(np.float)
        sigma_fd_raw = raw_dataset[4].data.astype(np.float)
        # If requested, kakes it small, to save memory
        if skip is not None:
            fd_raw = fd_raw[::skip]
            sigma_fd_raw = sigma_fd_raw[::skip]
        # Loads into the Dataset
        super().__init__(data=fd_raw, error=sigma_fd_raw)

dset = FaradayDepthOppermann2012(skip=256)

Now we can load this into a Measurements and Covariances objects

In [ ]:
dset.key

In [ ]:
from imagine import Measurements, Covariances

mea = Measurements()
cov = Covariances()

mea.append(dataset=dset)
cov.append(dataset=dset)

## Simulator

[Simulator objects](https://imagine-code.readthedocs.io/en/latest/imagine.simulators.html)
are responsible for converting into Observables the physical quantities computed/stored by the Field objects. 


In [ ]:
unit = 1/(u.GeV*u.m**2*u.s*u.steradian) 
unit

In [ ]:
unit * (4*np.pi*u.steradian) / c.c 
